<a href="https://colab.research.google.com/github/grillinr/evolutionary-computing/blob/main/final/final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

deep_learning_course_midterm_path = kagglehub.competition_download('deep-learning-course-midterm')

print('Data source import complete.')


# Import libraries and seed for easier checking

In [ ]:
# 0.66 is baseline for score
import random
import os
import argparse

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import accuracy_score, fbeta_score
from sklearn.model_selection import train_test_split

SEED = 5173
device = torch.device("cpu")

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Define helper functions

In [ ]:
def prepare_data(data, device):
    # Drop non-numeric and class columns
    X = data.drop(columns=["id", "record", "type"]).values.astype(np.float32)

    # Convert class to numeric value from 0-4
    y = data["type"].astype("category").values

    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y, dtype=torch.long).to(device)

    return X_tensor, y_tensor


def evaluate(model, device, data, criterion):
    model.eval()

    X, y = prepare_data(data, device)

    with torch.no_grad():
        logits = model(X)
        loss = criterion(logits, y)
        probs = torch.softmax(logits, dim=1).cpu().numpy()

    y_true = y.cpu().numpy()
    y_pred = probs.argmax(axis=1)

    acc = accuracy_score(y_true, y_pred)
    f_beta_m = fbeta_score(y_true, y_pred, average="macro", beta=2)

    return {
        "loss": loss.item(),
        "accuracy": acc,
        "f_beta_macro": f_beta_m,
    }

# Create Model Architecture (DNN)

In [ ]:
class DNN(nn.Module):
    def __init__(self, input_size=32, hidden=(32, 16, 8), num_classes=5, p=0.5):
        super().__init__()
        layers = []
        input_dim = input_size

        for h in hidden:
            layers.append(nn.Linear(input_dim, h))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(p))
            input_dim = h

        layers.append(nn.Linear(input_dim, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# Main Training loop

In [ ]:
# Load data
dataset = pd.read_csv("/kaggle/input/deep-learning-course-midterm/train.csv")
train_dataset, test_dataset = train_test_split(dataset, train_size=0.7, random_state=SEED)
# test_dataset = pd.read_csv("test.csv")

In [ ]:
# Configuration
lr = 1e-3
epochs = 100
hidden = (32, 16, 8)
p = 0.5

# Create model
model = DNN(hidden=hidden, p=p).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Prepare tensors
X_train, y_train = prepare_data(train_dataset, device)
y_train.min()

In [ ]:
train_loss = train_one_epoch(model, X_train, y_train, optimizer, criterion)

In [ ]:
# Training loop
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    out = model(X_train)
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()
    train_loss = loss.item()

    train_metrics = evaluate(model, device, train_dataset, criterion)
    val_metrics = evaluate(model, device, test_dataset, criterion)

    print(
        f"Epoch {epoch}/{epochs} | "
        f"train_loss={train_loss:.4f} train_acc={train_metrics['accuracy']:.4f} "
        f"train_f1={train_metrics['f_beta_macro']:.4f} | "
        f"val_loss={val_metrics['loss']:.4f} val_acc={val_metrics['accuracy']:.4f} "
        f"val_f1={val_metrics['f_beta_macro']:.4f}"
    )

# Test output on split data to avoid submitting

In [ ]:
X_test, y_test = prepare_data(test_dataset, device)

# Get predictions
with torch.no_grad():
    logits = model(X_test)
    probs = torch.softmax(logits, dim=1).cpu().numpy()
    predictions = probs.argmax(axis=1)

y_true = y_test.cpu().numpy()

# Calculate metrics
accuracy = accuracy_score(y_true, predictions)
f_beta = fbeta_score(y_true, predictions, average="macro", beta=2)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F-Beta (macro): {f_beta:.4f}")

In [ ]:
# Model stats evaluation
import torch
import torch.nn as nn
import json

def estimate_flops(model, input_shape):
    """
    Estimate FLOPs for Linear and Conv2d layers only.
    Args:
        model (nn.Module): PyTorch model
        input_shape (tuple): shape of one input sample, e.g., (1, 3, 224, 224) or (1, input_dim)
    Returns:
        total_flops (int)
    """
    flops = 0

    def count_layer(layer, x_in, x_out):
        nonlocal flops
        # Conv2d FLOPs = Kx * Ky * Cin * Cout * Hout * Wout
        if isinstance(layer, nn.Conv2d):
            out_h, out_w = x_out.shape[2:]
            kernel_ops = layer.kernel_size[0] * layer.kernel_size[1]
            flops += kernel_ops * layer.in_channels * layer.out_channels * out_h * out_w
        # Linear FLOPs = input_features * output_features
        elif isinstance(layer, nn.Linear):
            flops += layer.in_features * layer.out_features

    hooks = []
    for layer in model.modules():
        if isinstance(layer, (nn.Conv2d, nn.Linear)):
            hooks.append(layer.register_forward_hook(count_layer))

    dummy = torch.randn(input_shape)
    with torch.no_grad():
        model(dummy)

    for h in hooks:
        h.remove()

    return flops


def save_model_stats(model, input_dim, save_path="model_stats.json"):
    """
    Save parameter count and estimated FLOPs to JSON (no thop needed).
    Args:
        model (nn.Module): PyTorch model
        input_dim (int or tuple): Input feature dimension or shape
        save_path (str): Output JSON path
    """
    # Create dummy input shape
    input_shape = (1, input_dim) if isinstance(input_dim, int) else (1, *input_dim)

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())

    # Estimate FLOPs
    flops = estimate_flops(model, input_shape)

    # Format into M (millions)
    stats = {
        "flops": f"{flops/1e6:.3f}M",
        "num_parameters": f"{total_params/1e6:.3f}M"
    }
    return stats